In [1]:
!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

--2024-10-15 16:25:15--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: 'bank+marketing.zip'

     0K .......... .......... .......... .......... ..........  136K
    50K .......... .......... .......... .......... ..........  567K
   100K .......... .......... .......... .......... ..........  541K
   150K .......... .......... .......... .......... .......... 4.40M
   200K .......... .......... .......... .......... ..........  302K
   250K .......... .......... .......... .......... ..........  276K
   300K .......... .......... .......... .......... ..........  181M
   350K .......... .......... .......... .......... ..........  165M
   400K .......... .......... .......... .......... ..........  173M
   450K .......... ..........

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [156]:
df=pd.read_csv('bank/bank-full.csv',sep=';')

In [157]:
df.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no
45210,37,entrepreneur,married,secondary,no,2971,no,no,cellular,17,nov,361,2,188,11,other,no


In [158]:
df.columns = df.columns.str.lower().str.replace(' ','_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ','_')

In [159]:
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [160]:
numerical = ['age','balance','duration','pdays','previous','day','campaign']
categorical = ['job','marital','education','housing','contact','month','poutcome','y']

In [161]:
df_full = df[numerical+categorical]


In [162]:
df_full.isnull().sum()

age          0
balance      0
duration     0
pdays        0
previous     0
day          0
campaign     0
job          0
marital      0
education    0
housing      0
contact      0
month        0
poutcome     0
y            0
dtype: int64

## Question 1

In [163]:
df_full.education.value_counts()

education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64

R: The answer is 'secondary'.

## Question 2

In [164]:
from sklearn.model_selection import train_test_split

In [165]:
df_full.y=(df_full.y == 'yes').astype(int)

C:\Users\bruno\AppData\Local\Temp\ipykernel_25656\3201066553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full.y=(df_full.y == 'yes').astype(int)


In [166]:
df_full.y

0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int32

In [167]:
df_full_train, df_test = train_test_split(df_full, test_size=0.2, random_state=42)

In [168]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [169]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [170]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

In [171]:
del df_train['y']
del df_val['y']
del df_test['y']

In [172]:
df_full[numerical].corr()

,age,balance,duration,pdays,previous,day,campaign
age,1.000000,0.097783,-0.004648,-0.023758,0.001288,-0.009120,0.004760
balance,0.097783,1.000000,0.021560,0.003435,0.016674,0.004503,-0.014578
duration,-0.004648,0.021560,1.000000,-0.001565,0.001203,-0.030206,-0.084570
pdays,-0.023758,0.003435,-0.001565,1.000000,0.454820,-0.093044,-0.088628
previous,0.001288,0.016674,0.001203,0.454820,1.000000,-0.051710,-0.032855
day,-0.009120,0.004503,-0.030206,-0.093044,-0.051710,1.000000,0.162490
campaign,0.004760,-0.014578,-0.084570,-0.088628,-0.032855,0.162490,1.000000


R: pdays and previous

## Question 3

In [173]:
from sklearn.metrics import mutual_info_score

In [175]:
def mutual_info_y_score(series):
    return mutual_info_score(series, df_full_train.y)

In [176]:
mi = df_full_train[categorical].apply(mutual_info_y_score)

In [179]:
mi.sort_values(ascending=False)

y            0.359020
poutcome     0.029257
month        0.024774
contact      0.014164
housing      0.009800
job          0.007765
education    0.002458
marital      0.002019
dtype: float64

R:'poutcome'

## Question 4

In [180]:
from sklearn.feature_extraction import DictVectorizer

In [183]:
categorical = ['job','marital','education','housing','contact','month','poutcome']

In [190]:
train_dicts=df_train[categorical+numerical].to_dict(orient='records')

In [191]:
dv=DictVectorizer(sparse=False)


In [192]:
dv.fit(train_dicts)

DictVectorizer(sparse=False)

In [193]:
X_train = dv.fit_transform(train_dicts)

In [194]:
val_dicts = df_val[categorical+numerical].to_dict(orient='records')

In [195]:
X_val = dv.transform(val_dicts)

In [196]:
from sklearn.linear_model import LogisticRegression

In [206]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [207]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [208]:
model.predict_proba(X_val)

array([[0.9874201 , 0.0125799 ],
       [0.99060425, 0.00939575],
       [0.85291412, 0.14708588],
       ...,
       [0.95156803, 0.04843197],
       [0.99118824, 0.00881176],
       [0.72076348, 0.27923652]])

In [209]:
y_pred = model.predict_proba(X_val)[:,1]

In [210]:
y_decision = (y_pred >= 0.5)

In [211]:
(y_val == y_decision).mean().round(1)

0.9

In [ ]:
original_model_acc = 

R: Accuracy = 0.9

## Question 5

In [212]:
all_features = ['job','marital','education','housing','contact','month','poutcome','age','balance','duration','pdays','previous','day','campaign']

In [213]:
features_age= ['job','marital','education','housing','contact','month','poutcome','balance','duration','pdays','previous','day','campaign']

In [214]:
# REMOVING AGE
train_dicts=df_train[features_age].to_dict(orient='records')
dv=DictVectorizer(sparse=False)
dv.fit(train_dicts)
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[features_age].to_dict(orient='records')
X_val = dv.transform(val_dicts)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:,1]
y_decision = (y_pred >= 0.5)
(y_val == y_decision).mean()

0.9002433090024331

In [215]:
features_balance = ['job','marital','education','housing','contact','month','poutcome','age','duration','pdays','previous','day','campaign']

In [216]:
# REMOVING BALANCE
train_dicts=df_train[features_balance].to_dict(orient='records')
dv=DictVectorizer(sparse=False)
dv.fit(train_dicts)
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[features_balance].to_dict(orient='records')
X_val = dv.transform(val_dicts)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:,1]
y_decision = (y_pred >= 0.5)
(y_val == y_decision).mean()

0.9006856890068569

In [217]:
features_marital = ['job','education','housing','contact','month','poutcome','age','balance','duration','pdays','previous','day','campaign']

In [218]:
# REMOVING MARITAL
train_dicts=df_train[features_marital].to_dict(orient='records')
dv=DictVectorizer(sparse=False)
dv.fit(train_dicts)
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[features_marital].to_dict(orient='records')
X_val = dv.transform(val_dicts)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:,1]
y_decision = (y_pred >= 0.5)
(y_val == y_decision).mean()

0.9012386640123866

In [219]:
features_previous = ['job','marital','education','housing','contact','month','poutcome','age','balance','duration','pdays','day','campaign']

In [220]:
# REMOVING PREVIOUS
train_dicts=df_train[features_previous].to_dict(orient='records')
dv=DictVectorizer(sparse=False)
dv.fit(train_dicts)
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[features_previous].to_dict(orient='records')
X_val = dv.transform(val_dicts)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:,1]
y_decision = (y_pred >= 0.5)
(y_val == y_decision).mean()

0.9013492590134926

R: Removing 'age' has the least impact.

## Question 6

In [221]:
all_features = ['job','marital','education','housing','contact','month','poutcome','age','balance','duration','pdays','previous','day','campaign']

In [222]:
train_dicts=df_train[all_features].to_dict(orient='records')
dv=DictVectorizer(sparse=False)
dv.fit(train_dicts)
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[all_features].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [223]:
model = LogisticRegression(solver='liblinear', C=0.01, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:,1]
y_decision = (y_pred >= 0.5)
(y_val == y_decision).mean()

0.8980314089803141

In [224]:
model = LogisticRegression(solver='liblinear', C=0.1, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:,1]
y_decision = (y_pred >= 0.5)
(y_val == y_decision).mean()

0.9013492590134926

In [225]:
model = LogisticRegression(solver='liblinear', C=1, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:,1]
y_decision = (y_pred >= 0.5)
(y_val == y_decision).mean()

0.9015704490157045

In [226]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:,1]
y_decision = (y_pred >= 0.5)
(y_val == y_decision).mean()

0.9009068790090687

In [227]:
model = LogisticRegression(solver='liblinear', C=100, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:,1]
y_decision = (y_pred >= 0.5)
(y_val == y_decision).mean()

0.9011280690112807

R: C=1 best accuracy.